In [ ]:
!pip install -q langchain langchain-google-genai faiss-cpu pypdf langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


ERROR: Could not find a version that satisfies the requirement pil (from versions: none)
ERROR: No matching distribution found for pil


In [ ]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
from IPython.display import display,Markdown

In [ ]:
from google.colab import userdata
import os
api = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=api

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser
from langchain.prompts import ChatPromptTemplate

In [50]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

def get_llm(model_name):
    if model_name == "gemini-2.0-flash":
        llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash")
    else:
        llm = ChatGoogleGenerativeAI(model="models/gemini-pro-vision")
    return llm

model = get_llm("gemini-2.0-flash")
response = model.invoke("5 facts about Pakistan")
print(response.content)


Okay, here are 5 interesting facts about Pakistan:

1.  **World's Highest Paved International Road:** The Karakoram Highway, also known as the "Friendship Highway," connects Pakistan and China and is the highest paved international road in the world. It's an engineering marvel and offers stunning mountain scenery.

2.  **Home to K2:** Pakistan is home to K2, the second-highest mountain in the world after Mount Everest. It's known as the "Savage Mountain" due to its treacherous climbing conditions.

3.  **One of the Largest Irrigation Systems:** Pakistan has one of the largest contiguous irrigation systems in the world, fed by the Indus River and its tributaries. This system is crucial for the country's agriculture.

4.  **Youngest Nobel Prize Laureate:** Malala Yousafzai, a Pakistani activist for female education, became the youngest Nobel Prize laureate when she was awarded the Nobel Peace Prize in 2014 at the age of 17.

5.  **Edible Salt Mines:** The Khewra Salt Mine is the second l

In [39]:
# for checking text model
model([
    HumanMessage(content="answer with simple yes or no. is apple red?")
]).content



'Yes'

In [106]:
# for checking image model
def get_image(url,filename,extension):
  content = requests.get(url).content
  with open(f"/content/{filename}.{extension}","wb") as f:
    f.write(content)
  image = Image.open(f"/content/{filename}.{extension}")
  image.show()
  return image

In [107]:
image = get_image("https://earthshotprize.org/wp-content/uploads/2023/05/bee-on-flower.jpg",
                  "shoe_image","png")

In [108]:
# Gemini (and other multimodal models) expects an image URL or base64 image in a very specific format.
# This function ensures that your local or in-memory PIL image is converted into exactly what the model requires.
import base64
from io import BytesIO
def pil_image_to_base64_url(pil_image, format="JPEG"):
    buffered = BytesIO()     # It creates an in-memory binary stream — like a fake file stored in RAM. it store image in memory
    pil_image.save(buffered, format=format)  # This saves the PIL image into the BytesIO buffer as if it's writing to a file
    # Base64 is a way of encoding binary data (like images) into a text format
    # buffered.getvalue() gets the raw binary image data from the buffer.
    # .decode() turns the base64 bytes into a regular UTF-8 string.
    img_base64 = base64.b64encode(buffered.getvalue()).decode()
    return f"data:image/{format.lower()};base64,{img_base64}"

image = pil_image_to_base64_url(image,"png")


In [79]:
message = HumanMessage(
    content=[
        {
            "type":"image_url",
            "image_url":image
        }
    ]
)
model=get_llm("models/gemini-pro-vision")
model.invoke([message]).content     # it require a list of message

"The image is a close-up, macro photograph of a bumblebee (likely a common eastern bumblebee or a similar species) on a bright yellow flower.  The focus is sharply on the bee, showcasing its fuzzy body, black and yellow stripes, and its delicate legs and antennae. The bee appears to be gathering pollen or nectar.\n\nThe flower, likely a type of daisy or sunflower, is a vibrant, rich yellow. The central disc florets where the bee is perched are in sharp focus, creating a textured, almost velvety appearance.  The petals of the flower, extending outwards, are slightly blurred, creating a soft, bokeh effect.\n\nThe background is also blurred, with a gradient of yellow and darker tones, suggesting other flowers or foliage out of focus. This blurring helps to isolate the bee and the flower it's on, drawing the viewer's attention to the main subject.  The overall impression is one of warmth, brightness, and nature's intricate beauty."

In [86]:
path = "/content/a-introduction-bees-july2013.pdf"
def doc_loader(path_to_pdf):
  loader = PyPDFLoader(path_to_pdf)
  return loader.load()

doc = doc_loader(path)

In [87]:
def text_splitter(doc):
  splitter = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=65)
  return splitter.split_documents(doc)
chunks = text_splitter(doc=doc)

In [88]:
embedding_model = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vector_store = FAISS.from_documents(documents=chunks,
                                    embedding=embedding_model)

In [90]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [95]:
context = retriever.invoke("type of bee")

def context_format(context):
  return "\n\n".join(doc.page_content for doc in context)


In [96]:
text_llm = get_llm("gemini-2.0-flash")
image_model = get_llm("text_model")

In [98]:
prompt = ChatPromptTemplate.from_template(
    template = """{context}
          {question}
  Provide brief information and type of bee if possible from the context.
    """
)

In [103]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "context": retriever | RunnableLambda(context_format),
    "question":RunnablePassthrough()
})

final_chain_text = parallel_chain | prompt | text_llm | parser


In [104]:
result = final_chain.invoke("type of bees")
display(Markdown(result))

Here's a breakdown of the bee species mentioned in the context, along with information provided:

*   **A. mellifera (European Honey Bee):**
    *   Medium-sized workers.
    *   Cavity-nesting (builds nests in hollow spaces).
    *   Found almost worldwide.
    *   24 races exist.
    *   Some subspecies are important for managed beekeeping.
    *   *A.m. scutellata* (African bee) is a subspecies known for aggressive behavior.

*   **A. cerana (Asiatic Honey Bee):**
    *   Medium-sized workers.

*   **A. nigrocincta (Celebes Honey Bee):**
    *   Medium-sized workers.

*   **A. koschevnikovi (Koschevnikov's Honey Bee):**
    *   Medium-sized workers.

*   **A. nuluensis (Mount Nulu Honey Bee):**
    *   Medium-sized workers.

*   **A. florea (Red Dwarf Honey Bee):**
    *   Dwarf bee species.
    *   Builds single-comb open nests.

*   **A. andreniformis (Black Dwarf Honey Bee):**
    *   Dwarf bee species.
    *   Builds single-comb open nests.

*   **A. dorsata (Giant Honey Bee):**
    *   Giant bee species.
    *   Builds single-comb open nests.

*   **A. laboriosa (Himalayan Honey Bee):**
    *   Giant bee species.
    *   Builds single-comb open nests.

In [105]:
full_chain = RunnablePassthrough() | image_model | parser | final_chain_text

In [118]:
from langchain_core.messages import HumanMessage
from IPython.display import display, Markdown

message = HumanMessage(content=[
    {
        "type": "text",
        "text": "Provide information about the image from the context only"
    },
    {
        "type": "image_url",
        "image_url": image  # make sure this is a valid base64 or hosted URL
    }
])

final_result = full_chain.invoke([message])
display(Markdown(final_result))


Based on the text and the image:

*   **Type of Bee:** Bumblebee (as described in the image caption)
*   **Information:**
    *   Honey bees are important pollinators.
    *   Bee colonies are complex societies where individual bees rely on each other to survive.